In [2]:
import numpy as np

-1 Group the spectrum

In [ ]:
for f in *-fovt.pi
do
export name=$(basename "$f" -fovt.pi)
ftgrouppha infile=$f outfile=${name}_opt.pi respfile=${name}.rmf grouptype=opt
done

0 load the data

In [ ]:
data 1:1 mos1S001_opt.pi
data 2:2 mos2S002_opt.pi

res 1 mos1S001.rmf
res 2 mos2S002.rmf

arf 1 mos1S001.arf
arf 2 mos2S002.arf

backgrnd 1 mos1S001-bkg.pi	
backgrnd 2 mos2S002-bkg.pi


setp energy
statistic cstat
abun aspl

ign 1:0.0-0.3,11.0-** 
ign 2:0.0-0.3,11.0-**

# ignore instrumental lines band
ign 1:1.2-1.8 
ign 2:1.2-1.8
ign bad


cpd /xs
pl ldat

1 get backscal

In [3]:
# get backscal in the pi files
from astropy.io import fits
import os

# homepath = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/data/ID3460/reduction/SDSSTG3460/SDSSTG3460_bkg'
homepath = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/data/ID3460/fitting/reg1'

os.chdir(homepath)

# every pixel length is 0.05 arcsec

for name in ['mos1S001', 'mos2S002']:
    f = fits.open(f'{name}-bkg.pi')
    print(f'''{name}: {np.round(f[1].header['BACKSCAL'] * (0.05/60) ** 2, 3)}''')

mos1S001: 0.82
mos2S002: 0.834


3 set bkg mdl

In [ ]:
# set bkg model
# >bkg_mdl.xcm 
mo const*const*(apec + tbabs*(apec+apec+pow))

In [ ]:
# extract region area
new 1 0.82
new 18 0.834

# rosat spectrum has been normalized to 1 arcmin2 ? yes!
free 1,18

# inst variance
new 2 1
new 19 1
free 2,19

# LHB
new 3 0.11
new 6 7.98669E-07
free 3,6

# nH
new 7 0.0243
free 7

# GH_hard
# not always need to consider
new 8 0.19156
new 11 1.53306E-06
free 8,11

# ICM
# norm can be spatially variable
new 12 1
new 15 1e-5
new 13 0.3
new 14 0.043 


# cxb
new 16 1.46
new 17 7.65062E-07
free 16,17

#fix abun first. thaw later
thaw 13

5 set spf mdl
don't set if the spf norm is 0 !!

In [ ]:
resp 2:1 mos1-diag.rsp
resp 2:2 mos2-diag.rsp 

backgrnd 2:1 mos1S001-bkg.pi	
backgrnd 2:2 mos2S002-bkg.pi

model 2:spf const*const*(bknpow)

In [ ]:
# set area const
new spf:1=1
new spf:7=18
new spf:2=2
new spf:8=19

# set bknpow index
new spf:3 0.7
new spf:9 0.7
new spf:5 0.7
new spf:11 0.7
free spf:3,9,5,11

# set breakE
free spf:4

# estimate spf norm
new spf:6 1e-5
new spf:12 1e-5

